In [33]:
import pandas as pd

In [34]:
df = pd.read_csv('/content/sample_data/papers.csv',quoting=3,on_bad_lines='warn')

Streaming output truncated to the last 5000 lines.
Skipping line 4129038: expected 7 fields, saw 10
Skipping line 4129047: expected 7 fields, saw 14
Skipping line 4129138: expected 7 fields, saw 9
Skipping line 4129331: expected 7 fields, saw 8
Skipping line 4129362: expected 7 fields, saw 13
Skipping line 4129389: expected 7 fields, saw 11
Skipping line 4129497: expected 7 fields, saw 8
Skipping line 4129815: expected 7 fields, saw 10
Skipping line 4129821: expected 7 fields, saw 10
Skipping line 4129823: expected 7 fields, saw 16
Skipping line 4129825: expected 7 fields, saw 8
Skipping line 4129837: expected 7 fields, saw 14
Skipping line 4129841: expected 7 fields, saw 14
Skipping line 4129844: expected 7 fields, saw 10
Skipping line 4129847: expected 7 fields, saw 16
Skipping line 4129849: expected 7 fields, saw 8
Skipping line 4129851: expected 7 fields, saw 16
Skipping line 4129857: expected 7 fields, saw 8
Skipping line 4129872: expected 7 fields, saw 8
Skipping line 4129887: ex

In [35]:
df.head()
df

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,"""767"
1,SELF-ORGANIZATION OF ASSOCIATIVE DATABASE,NaN,NaN,NaN,NaN,NaN,NaN
2,AND ITS APPLICATIONS,NaN,NaN,NaN,NaN,NaN,NaN
3,Hisashi Suzuki and Suguru Arimoto,NaN,NaN,NaN,NaN,NaN,NaN
4,Osaka University,Toyonaka,Osaka 560,Japan,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4298640,[11] K. Waters and D. Terzopoulos. Modeling an...,NaN,NaN,NaN,NaN,NaN,NaN
4298641,The Journal of Visualization and Computer Anim...,2: 123-128,1991.,NaN,NaN,NaN,NaN
4298642,[12] L. Williams. Performance-driven facial an...,NaN,NaN,NaN,NaN,NaN,NaN
4298643,Proceedings,24(4):235-242,1990.,NaN,NaN,NaN,NaN


In [36]:
df.shape

(4298645, 7)

In [37]:
df = df.iloc[:5000,:]

In [38]:
df.shape

(5000, 7)

In [39]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [40]:
stop_words = set(stopwords.words('english'))

Creating a list of custom stop words

In [41]:
new_words = ["fig","figure","image","sample","using","show","result","large","also","one","two","three","four","five","seven","eight","nine"]

In [42]:
stop_words = list(stop_words.union(new_words))

In [43]:
def preprocess_text(text):
  if isinstance(text, float):
    text = str(text)
  text = text.lower()
  text = re.sub(r'<.*?>',' ',text)
  text = re.sub(r'[^a-zA-Z]',' ',text)
  text = nltk.word_tokenize(text)
  text = [word for word in text if word not in stop_words]
  text = [word for word in text if len(word) >= 3]
  stemming = PorterStemmer()
  text = [stemming.stem(word) for word in text]
  return ' '.join(text)

In [44]:
preprocess_text("tHiS iS to learning and understanding 4545 %$# PyThoN <h1><p> web language</p></h1>")

'learn understand python web languag'

In [45]:
docs = df['paper_text'].apply(lambda x: preprocess_text(x))

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_df = 0.95, max_features = 5000, ngram_range = (1,3))
word_count_vector = cv.fit_transform(docs)

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfTransformer = TfidfTransformer(use_idf=True,smooth_idf=True)
tfidfTransformer.fit(word_count_vector)

TfidfTransformer()

In [48]:
feature_names = cv.get_feature_names_out()

In [49]:
def get_keywords(idx,docs):
  docs_word_count = tfidfTransformer.transform(cv.transform([docs[idx]]))
  #build sparse matrix
  docs_word_count = docs_word_count.tocoo()
  tuples = zip(docs_word_count.col,docs_word_count.data)
  sorted_items = sorted(tuples,key = lambda x: (x[1],x[0]),reverse=True)

  sorted_items = sorted_items[:10]
  score_vals = []
  feature_vals = []
  for idx, score in sorted_items:
    score_vals.append(round(score,3))
    feature_vals.append(feature_names[idx])

  results = {}
  for idx in range(len(feature_vals)):
    results[feature_vals[idx]] = score_vals[idx]
  return results

def print_keywords(idx,keywords,idf):
  print("\n--------Title-------")
  print(df['title'][idx])
  print("\n--------Abstract-------")
  print(df['abstract'][idx])
  print("\n--------Keywords-------")
  for k in keywords:
    print(k,keywords[k])

idx = 350
keywords = get_keywords(idx,docs)
print_keywords(idx,keywords,docs)



--------Title-------
nan

--------Abstract-------
nan

--------Keywords-------


In [50]:
#creating the pickle files
import pickle
pickle.dump(tfidfTransformer,open('tfidfTransformer.pkl','wb'))
pickle.dump(cv,open('count_vector.pkl','wb'))
pickle.dump(feature_names,open('feature_names.pkl','wb'))